In [ ]:
from pathlib import Path
from ultralytics import YOLO
import numpy as np
from  ultralytics.engine.results import Boxes as Boxes
from typing import List

In [ ]:
model = YOLO("best_yolov8m_tune_10_iter.pt")

In [ ]:

def make_dict(image_path: Path, box: Boxes):
    x, y, w, h = box.xywhn[0]

    line = {'filename': image_path.name, 
            'class_id': int(box.cls), 
            'rel_x': x,
            'rel_y': y,
            'width': w,
            'height': h}
    return line

def process_boxes(image_path: Path, np_boxes: List[Boxes], threshold: float):
    lines = []
    for box in np_boxes:
        if box.conf > threshold:
            line = make_dict(image_path, box)    
            lines.append(line)
    return lines

In [ ]:

def get_metrics_for_file(image: Path, threshold: float):
    res = model(image)[0]
    lines = process_boxes(image, res.cpu().numpy().boxes, threshold=threshold)
    return lines

In [ ]:
def get_metrics_for_file_array(images: List[Path], threshold: float):
    results = []
    for image in images:
        lines, confs = get_metrics_for_file(image, threshold=threshold)
        results += lines
    return results

In [ ]:
test_files_folder = Path("test_dataset/")
threshold = 0.75
test_images = list(test_files_folder.glob("*.jpg"))
test_results, confss = get_metrics_for_file_array(test_images, threshold=threshold);


In [ ]:
import csv

test_results_filename = Path("yolo8m_tuned.csv")

with open(test_results_filename, 'w') as f:

    writer = csv.DictWriter(f, fieldnames=test_results[0].keys(),delimiter=';')
    writer.writeheader()
    writer.writerows(test_results)
    